#### 감성분석 -> 머신러닝
    - 데이터셋 : 전처리
    - BoW 모델
        - 단어를 특성 벡터로 변환
        - tf-idf 단어 적합성 평가
        - 텍스트 데이터 정제
        - 문서를 토큰으로 나누기
    - LogisticRegession

In [1]:
# 데이터 셋
# http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [2]:
import pandas as pd
from glob import glob
file_lists = glob('C:\\python_src2\\data\\movie\\train\\neg\\*.txt')
pd_lists = []
for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_neg_df = pd.concat(pd_lists,ignore_index=True)
train_neg_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0
3,"Sorry everyone,,, I know this is supposed to b...",0
4,When I was little my parents took me along to ...,0


In [3]:
# postive  동일하게   train_pos_df
# train_df = pd.concat([train_nag_df, train_pos_df])
# movie_data.csv로 저장

file_lists = glob('C:\\python_src2\\data\\movie\\train\\pos\\*.txt')
pd_lists = []
for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_pos_df = pd.concat(pd_lists,ignore_index=True)
train_pos_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [4]:
train_df = pd.concat([ train_pos_df,  train_neg_df ],ignore_index=True)
train_df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [5]:
train_df.to_csv("movie_data.csv", index=False, encoding='utf-8')

In [6]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


In [7]:
# BoW(Bag of Words) 모델
# 문자를 숫자벡터
# 단어의 등장횟수를 카운트

# 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전
# 각 문서(리뷰데이터)를 사전을 기준으로 벡터화  N번째단어가 문서에서 3번나오면 벡터의 N번째값이 3이 된다.
# 문서1 : "나는 영화가 좋다"
# 문서2 : "나는 영화가 싫다"
# 사전 : {'나는':0, '영화가':1,'좋다':2,'싫다':3}
# 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게
# 문서1벡터 : [0,1,2] - > [1,1,1,0]
# 문서2벡터 : [0,1,3] - > [1,1,0,1]

In [8]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    "The sun is shining",
    "The weather is sweet"
])
bag = count.fit_transform(docs)

In [9]:
count.vocabulary_

{'the': 4, 'sun': 2, 'is': 0, 'shining': 1, 'weather': 5, 'sweet': 3}

In [10]:
# 문서 d에 등장한 단어 t의 횟수를 tf(t,d)
# BoW를 보완하면서 좀더 정교환 텍스트 벡터화 방식  TF-IDF(Term Frequency -Inverse Document Frequency)
# TF : 특정문서에서 자주 등장하는 단어
# IDF : 전체문서에 드물게 등장하는 단어
# 특정문서에서 자주 등장하지만 전체 문장에서 드물게 등장하는 단어에 높은 가중치를 부여 - 그 문장을 잘 대표하는 핵심 단어를 찾는다
# TF(t,d)  단어 t가 문장 d에 나타난 횟수 / 문서d의 모든 단어수
# IDF(t,D) : log( 총 문서수|D| / 단어 t를 포함한 문서의수 df(t) )  -- log 단어의 희귀성을 너무 과하게 반영하지 않도록 스케일링
# 분모에 + 1(사이킷런의 경우) :  분모가 0되는 것을 방지
# log( 1+ |D| / 1+df(f)  )

#TF-IDF(t,d,D) = TF(t,d) x IDF(t,D)

# "나는"
# TF : 리뷰에 3번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 9000개 나옴(매우 낮음)
# tf-idf  높음 x 매우낮음 = 낮음(중요도가 낮음)

# "명작" TF : 리뷰에 2번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 50개 나옴(매우 높음)
# tf-idf  높음 x 매우높음 = 높음(핵심단어)


In [11]:
df.review[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [12]:
# 데이터 정재... html tag 와 같은 불필요한 string이 보임... 특수기호 기타등등.  < - ' 
import re
def preprocessor(s):
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', s)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    return clean

In [13]:
df['review'] = df.review.apply(preprocessor)

In [14]:
# 문서를 토큰으로 나누기
# %pip install nltk
# %conda install -c conda-forge nltk

In [21]:
from nltk.stem.porter import PorterStemmer

def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 어간추출 Stemming  단어의 접미사 -s -es -ing -ed 등등.. 를 강제로 제거해서 단어의 원형을 찾는과정
tokenizer(df.review[0][:100])

['Bromwell',
 'High',
 'is',
 'a',
 'cartoon',
 'comedy.',
 'It',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life,']

In [23]:
tokenizer('runners like running')

['runners', 'like', 'running']

In [ ]:
# 불용어
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')    # 불용어 사전 다운로드
stopwords.words('english')